# Аналіз набору даних новобудов у місті Київ

Для створення актуального набору даних із достатньою кількістю категоріальних критеріїв було створено скрапер за допомогою технології Puppeteer.

Дані збираються з веб-сайту lun.ua - найпопулярнішого агрегатору даних про новобудови в Україні.

Для аналізу набору даних використовується бібліотека pandas.

Ціллю збору даних є використання їх для тренування нейронної мережі яка моделює вартість заданого обʼєкту нерухомості із певними характеристками.

#### Словник

Характеристики обʼєкту (англ. Features) - характеристики з яких будуть обрані залежні та незалежні змінні для тренування моделі для знаходження залежності незалежних змінних від залежних.

Форма (англ. Shape) - описана через формат (N, M) де N - кількість рядків, M - кількість комірок

Теплова карта (англ. Heatmap) - формат візуалізації даних кореляції між характеристиками набору даних

#### Предметна Область

##### Залежні змінні/ Незалежні змінні

Залежною змінною є ціна на обʼєкт нерухомості.
Незалежними змінними є множина характеристик обʼєкту нерухомості що включає в себе числові значення та категоріальні дані.

## Ініціалазація бібліотеки Pandas

In [1]:
import pandas as pd
# бібліотека використовується для візуалізації характеристик набору даних та візуалізацій залежності між змінними
import matplotlib.pylab as plt
# бібліотека для візуалізації кореляції між характеристиками у вигляді теплової карти
import seaborn as sns
# використовується для перемішування набору даних для кращого ефекту від тренування моделі
import random
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Допоміжні бібліотеки

In [2]:
def df_to_list(df):
    return df.values.flatten().tolist()

## Приклад записів з набору даних

In [3]:
df = pd.read_csv('./real-estate.csv')
df.shape

(3459, 17)

In [4]:
df.head(3)

,residence,developer,attributes/class,attributes/construction_technology,attributes/walls,attributes/insulation,attributes/heating,attributes/num_apartments,attributes/state,attributes/protected_area,attributes/parking,financials/installmentPlan,financials/installmentPlanTerm,pricePerUnit,price,area,rooms
0,ЖК Montreal House,Alliance Novobud,бізнес,монолітно-каркасна,цегла,мінеральна вата,автономне (в будинку),328,без ремонту,від авто та сторонніх,підземний з ліфтом (187 паркомісць),True,1.0,116825.0,5023475.0,43,1
1,ЖК Montreal House,Alliance Novobud,бізнес,монолітно-каркасна,цегла,мінеральна вата,автономне (в будинку),328,без ремонту,від авто та сторонніх,підземний з ліфтом (187 паркомісць),True,1.0,116825.0,6308550.0,54,1
2,ЖК Montreal House,Alliance Novobud,бізнес,монолітно-каркасна,цегла,мінеральна вата,автономне (в будинку),328,без ремонту,від авто та сторонніх,підземний з ліфтом (187 паркомісць),True,1.0,116825.0,5958075.0,51,1


## Фільтрація набору даних

### Пусті значення

In [5]:
df.isna().sum()

residence                                0
developer                              426
attributes/class                        14
attributes/construction_technology       0
attributes/walls                         0
attributes/insulation                    0
attributes/heating                       0
attributes/num_apartments               10
attributes/state                         0
attributes/protected_area               10
attributes/parking                       0
financials/installmentPlan             572
financials/installmentPlanTerm        2204
pricePerUnit                          1276
price                                 1276
area                                     0
rooms                                    0
dtype: int64

#### Новобудови без ціни

In [6]:
df = df.loc[~df['price'].isna()]

#### Новобудови без розстрочки

In [7]:
df.loc[df['financials/installmentPlanTerm'].isnull(), 'financials/installmentPlanTerm'] = 0

#### Новобудови без вказаного забудовника

In [8]:
df.loc[df['developer'].isnull(), 'developer'] = 'Невідомий Забудовник'

#### Новобудови без інформації про кількість квартир у будинку

In [9]:
df = df.loc[~df['attributes/num_apartments'].isna()]

#### Результат фільтрації

In [10]:
df.isna().sum()

residence                             0
developer                             0
attributes/class                      0
attributes/construction_technology    0
attributes/walls                      0
attributes/insulation                 0
attributes/heating                    0
attributes/num_apartments             0
attributes/state                      0
attributes/protected_area             0
attributes/parking                    0
financials/installmentPlan            0
financials/installmentPlanTerm        0
pricePerUnit                          0
price                                 0
area                                  0
rooms                                 0
dtype: int64

In [11]:
df.shape

(2183, 17)

### NaN значення

In [12]:
df.isnull().sum()

residence                             0
developer                             0
attributes/class                      0
attributes/construction_technology    0
attributes/walls                      0
attributes/insulation                 0
attributes/heating                    0
attributes/num_apartments             0
attributes/state                      0
attributes/protected_area             0
attributes/parking                    0
financials/installmentPlan            0
financials/installmentPlanTerm        0
pricePerUnit                          0
price                                 0
area                                  0
rooms                                 0
dtype: int64

In [13]:
df = df.loc[~df['attributes/num_apartments'].isnull()]

#### Результат фільтрації

In [14]:
df.isnull().sum()

residence                             0
developer                             0
attributes/class                      0
attributes/construction_technology    0
attributes/walls                      0
attributes/insulation                 0
attributes/heating                    0
attributes/num_apartments             0
attributes/state                      0
attributes/protected_area             0
attributes/parking                    0
financials/installmentPlan            0
financials/installmentPlanTerm        0
pricePerUnit                          0
price                                 0
area                                  0
rooms                                 0
dtype: int64

In [15]:
df.shape

(2183, 17)

### Дублікати

In [16]:
df.loc[df.duplicated()]

,residence,developer,attributes/class,attributes/construction_technology,attributes/walls,attributes/insulation,attributes/heating,attributes/num_apartments,attributes/state,attributes/protected_area,attributes/parking,financials/installmentPlan,financials/installmentPlanTerm,pricePerUnit,price,area,rooms
4,ЖК Montreal House,Alliance Novobud,бізнес,монолітно-каркасна,цегла,мінеральна вата,автономне (в будинку),328,без ремонту,від авто та сторонніх,підземний з ліфтом (187 паркомісць),True,1.0,116825.0,6308550.0,54,1
12,ЖК Montreal House,Alliance Novobud,бізнес,монолітно-каркасна,цегла,мінеральна вата,автономне (в будинку),328,без ремонту,від авто та сторонніх,підземний з ліфтом (187 паркомісць),True,1.0,106750.0,9394000.0,88,2
13,ЖК Montreal House,Alliance Novobud,бізнес,монолітно-каркасна,цегла,мінеральна вата,автономне (в будинку),328,без ремонту,від авто та сторонніх,підземний з ліфтом (187 паркомісць),True,1.0,106750.0,9394000.0,88,2
28,ЖК LUCKY LAND,DIM,комфорт,монолітно-каркасна,керамоблок,мінеральна вата,індивідуальне (в квартирі),5183,"без ремонту, з ремонтом",від авто та сторонніх,"гостьовий (2500 паркомісць), підземний з ліфто...",True,3.0,66425.0,1859900.0,28,1
35,ЖК LUCKY LAND,DIM,комфорт,монолітно-каркасна,керамоблок,мінеральна вата,індивідуальне (в квартирі),5183,"без ремонту, з ремонтом",від авто та сторонніх,"гостьовий (2500 паркомісць), підземний з ліфто...",True,3.0,66425.0,3586950.0,54,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,ЖК Оберіг-2,Київміськбуд,комфорт,монолітно-каркасна,"цегла, газоблок (газобетон)",мінеральна вата,централізоване,820 (2 будинки),без ремонту,ні,"гостьовий, підземний",True,0.0,40550.0,1540900.0,38,1
2854,ЖК Оберіг-2,Київміськбуд,комфорт,монолітно-каркасна,"цегла, газоблок (газобетон)",мінеральна вата,централізоване,820 (2 будинки),без ремонту,ні,"гостьовий, підземний",True,0.0,40550.0,1581450.0,39,1
2866,ЖК Оберіг-2,Київміськбуд,комфорт,монолітно-каркасна,"цегла, газоблок (газобетон)",мінеральна вата,централізоване,820 (2 будинки),без ремонту,ні,"гостьовий, підземний",True,0.0,36450.0,2515050.0,69,2
2871,ЖК Оберіг-2,Київміськбуд,комфорт,монолітно-каркасна,"цегла, газоблок (газобетон)",мінеральна вата,централізоване,820 (2 будинки),без ремонту,ні,"гостьовий, підземний",True,0.0,36950.0,2771250.0,75,3


### Очистка від дублікатів

In [17]:
df = df.loc[~df.duplicated()].reset_index(drop=True).copy()

In [18]:
df.shape

(1827, 17)

### Приведення типів

In [19]:
df["attributes/num_apartments"] = pd.to_numeric(df["attributes/num_apartments"], errors='coerce')

In [20]:
df.dtypes

residence                              object
developer                              object
attributes/class                       object
attributes/construction_technology     object
attributes/walls                       object
attributes/insulation                  object
attributes/heating                     object
attributes/num_apartments             float64
attributes/state                       object
attributes/protected_area              object
attributes/parking                     object
financials/installmentPlan             object
financials/installmentPlanTerm        float64
pricePerUnit                          float64
price                                 float64
area                                    int64
rooms                                   int64
dtype: object

## Ознайомлення із набором даних

### Форма

In [21]:
df.shape

(1827, 17)

### Типи даних

In [22]:
df.dtypes

residence                              object
developer                              object
attributes/class                       object
attributes/construction_technology     object
attributes/walls                       object
attributes/insulation                  object
attributes/heating                     object
attributes/num_apartments             float64
attributes/state                       object
attributes/protected_area              object
attributes/parking                     object
financials/installmentPlan             object
financials/installmentPlanTerm        float64
pricePerUnit                          float64
price                                 float64
area                                    int64
rooms                                   int64
dtype: object

### Cтатистика

In [23]:
df.describe()

,attributes/num_apartments,financials/installmentPlanTerm,pricePerUnit,price,area,rooms
count,1452.000000,1827.000000,1827.000000,1.827000e+03,1827.000000,1827.000000
mean,1530.763085,1.295567,69474.603175,5.778772e+06,79.166940,2.099617
std,1879.105511,1.807311,40890.805075,5.886273e+06,36.437752,1.022172
min,9.000000,0.000000,23450.000000,2.187000e+05,3.000000,1.000000
25%,321.000000,0.000000,40525.000000,2.775825e+06,51.000000,1.000000
50%,724.000000,0.000000,57275.000000,4.092525e+06,74.000000,2.000000
75%,1898.000000,2.000000,83325.000000,6.800938e+06,98.000000,3.000000
max,8339.000000,7.000000,333700.000000,8.342500e+07,278.000000,5.000000


### Характеристики новобудов

#### Всі характеристики

In [24]:
pd.DataFrame(data=df.columns, columns=['Характеристка обʼєкту нерухомості'])

,Характеристка обʼєкту нерухомості
0,residence
1,developer
2,attributes/class
3,attributes/construction_technology
4,attributes/walls
5,attributes/insulation
6,attributes/heating
7,attributes/num_apartments
8,attributes/state
9,attributes/protected_area


In [25]:
def save_charateristics_mapping(c, filename):
    json_output = json.dumps(df_to_list(c), indent=2, ensure_ascii=False)

    with open(f"{filename}.json", "w", encoding='utf8') as outfile:
        outfile.write(json_output)

#### Назва житлового комплексу

In [26]:
residential_complex_names = pd.DataFrame(data=df.residence.unique(), columns=['Назва житлового комплексу'])
save_charateristics_mapping(residential_complex_names, 'residential_complex_names')
residential_complex_names

NameError: name 'json' is not defined

#### Забудовник

In [ ]:
developers = pd.DataFrame(data=df.developer.unique(), columns=['Забудовник'])
save_charateristics_mapping(developers, 'developers')
developers

#### Клас новобудови

In [ ]:
building_classes = pd.DataFrame(data=df['attributes/class'].unique(), columns=['Клас новобудови'])
save_charateristics_mapping(building_classes, 'building_classes')
building_classes

#### Технологія будівництва

In [ ]:
technologies = pd.DataFrame(data=df['attributes/construction_technology'].unique(), columns=['Технологія будівництва'])
save_charateristics_mapping(technologies, 'technologies')
technologies

#### Матеріал стін

In [ ]:
walls = pd.DataFrame(data=df['attributes/walls'].unique(), columns=['Матеріал стін'])
save_charateristics_mapping(walls, 'walls')
walls

#### Утелпення стін

In [ ]:
insulations = pd.DataFrame(data=df['attributes/insulation'].unique(), columns=['Утелпення стін'])
save_charateristics_mapping(insulations, 'insulations')
insulations

#### Опалення

In [ ]:
heating_options = pd.DataFrame(data=df['attributes/heating'].unique(), columns=['Опалення'])
save_charateristics_mapping(heating_options, 'heating_options')
heating_options

#### Стан ремонту

In [ ]:
renovation_states = pd.DataFrame(data=df['attributes/state'].unique(), columns=['Стан ремонту'])
save_charateristics_mapping(renovation_states, 'renovation_states')
renovation_states

#### Закрита територія

In [ ]:
protected_area_options = pd.DataFrame(data=df['attributes/protected_area'].unique(), columns=['Закрита територія'])
save_charateristics_mapping(protected_area_options, 'protected_area_options')
protected_area_options

#### Паркінг

In [ ]:
pd.DataFrame(data=df['attributes/parking'].unique(), columns=['Паркінг'])

#### Термін розстрочки

In [ ]:
ax = df['financials/installmentPlanTerm'].value_counts().head(10).plot(kind='bar', title='Термін розстрочки')

ax.set_xlabel('Років')
ax.set_ylabel('Пропозицій')

plt.show()

#### Ціна за квадратний метр

In [ ]:
ax = df.copy().sort_values(by='pricePerUnit')['pricePerUnit'].plot(kind='hist', bins=50, title='Ціна за квадратний метр')

ax.set_xlabel('Ціна (грн)')
ax.set_ylabel('Частота пропозиції')

plt.show()

#### Фільтрація цін за квадратний метр (приведення до нормального розподілу)

In [ ]:
df = df[~(df['pricePerUnit'] > 230_000)]

In [ ]:
ax = df.copy().sort_values(by='pricePerUnit')['pricePerUnit'].plot(kind='hist', bins=50, title='Ціна за квадратний метр')

ax.set_xlabel('Ціна (грн)')
ax.set_ylabel('Частота пропозиції')

plt.show()

#### Ціна новобудови

In [ ]:
ax = df.copy().sort_values(by='price')['price'].plot(kind='hist', bins=50, title='Ціна новобудови')

ax.set_xlabel('Ціна (грн)')
ax.set_ylabel('Частота пропозиції')

plt.show()

#### Фільтрація цін (приведення до нормального розподілу)

In [ ]:
df = df[~(df['price'] > 22000000)] # 22 мільйони гривень

In [ ]:
ax = df.copy().sort_values(by='price')['price'].plot(kind='hist', bins=50, title='Ціна новобудови')

ax.set_xlabel('Ціна (грн)')
ax.set_ylabel('Частота пропозиції')

plt.show()

#### Житлова площа

In [ ]:
ax = df.copy().sort_values(by='area')['area'].plot(kind='hist', bins=50, title='Житлова площа')

ax.set_xlabel('Житлова площа (метри квадратні)')
ax.set_ylabel('Частота пропозиції')

plt.show()

#### Кількість кімнат

In [ ]:
ax = df['rooms'].value_counts().head(10).plot(kind='bar', title='Кількість кімнат')

ax.set_xlabel('Кімнат')
ax.set_ylabel('Пропозицій')

plt.show()

#### Кількість квартир у новобудові

In [ ]:
ax = df.copy().sort_values(by='attributes/num_apartments')['attributes/num_apartments'].plot(kind='hist', bins=100, title='Кількість квартир у новобудові')

ax.set_xlabel('К-сть квартир у новобудові')
ax.set_ylabel('Частота пропозиції')

plt.show()

## Залежність між залежними та незалежними змінними (числові значення)

In [ ]:
sns.pairplot(df, vars=[
    'price',
    'attributes/num_apartments',
    'financials/installmentPlanTerm',
    'pricePerUnit',
    'area',
    'rooms'
], hue='attributes/construction_technology')

plt.show()

In [ ]:
racorr = df[[
    'price',
    'attributes/num_apartments',
    'financials/installmentPlanTerm',
    'pricePerUnit',
    'area',
    'rooms'
]].dropna().corr()

sns.heatmap(data=racorr, annot=True)

plt.show()

## Підготовка даних до використання у нейронній мережі для тренування

### Словник

Мітка (eng. Label) - кінцеве (eng. target) значення що є еталоном для тренувального набору даних

### Оцифровка нечислових значень

Для тренування нейронної мережі, значення всіх характеристик повинні мати числові значення для прорахунку похідної функції втрат при кожній з ітерацій тренувального процесу

In [ ]:
df.values[0]

#### Допоміжні функції

In [ ]:
# index + 1 використовується для того щоб позначити саме відсутні значення нулем
def apartment_to_numeric(apartment):
    return np.array([
        df_to_list(residential_complex_names).index(apartment[0]),
        df_to_list(developers).index(apartment[1]),
        df_to_list(building_classes).index(apartment[2]),
        df_to_list(technologies).index(apartment[3]),
        df_to_list(walls).index(apartment[4]),
        df_to_list(insulations).index(apartment[5]),
        df_to_list(heating_options).index(apartment[6]),
        # к-сть квартри в новобудові
        apartment[7],
        df_to_list(renovation_states).index(apartment[8]),
        df_to_list(protected_area_options).index(apartment[9]),
        # наявність паркінгу - 1/0
        int(bool(apartment[10])),
        # наявність розстрочки
        int(apartment[11]),
        # термін розстрочки в роках
        apartment[12],
#         # ціна за квадратний метр
#         apartment[13],
#         # ціна квартири
#         apartment[14],
        # площа квартири
        apartment[15],
        # к-сть кімнта
        apartment[16],
    ])

# list(map(apartment_to_numeric, df.values[:3]))

In [ ]:
def show_entities(num):
    pd.set_option('display.max_columns', num)
    pd.set_option('display.max_rows', num)

#### Процес оцифровки

In [ ]:
df_numeric = df.copy()

# словники для збереження відношення між строковими та числовими репрезентаціями значень
residence_map = {}
developer_map = {}
class_map = {}
construction_technology_map = {}
walls_map = {}
insulation_map = {}
heating_map = {}
state_map = {}
protected_area_map = {}
parking_map = {}
installmentPlan_map = {}

df_numeric['residence'] = df_numeric['residence'].map(lambda r: df_to_list(residential_complex_names).index(r))
df_numeric['developer'] = df_numeric['developer'].map(lambda d: df_to_list(developers).index(d))
df_numeric['attributes/class'] = df_numeric['attributes/class'].map(lambda c: df_to_list(building_classes).index(c))
df_numeric['attributes/construction_technology'] = df_numeric['attributes/construction_technology'].map(lambda c: df_to_list(technologies).index(c))
df_numeric['attributes/walls'] = df_numeric['attributes/walls'].map(lambda w: df_to_list(walls).index(w))
df_numeric['attributes/insulation'] = df_numeric['attributes/insulation'].map(lambda i: df_to_list(insulations).index(i))
df_numeric['attributes/heating'] = df_numeric['attributes/heating'].map(lambda h: df_to_list(heating_options).index(h))
df_numeric['attributes/state'] = df_numeric['attributes/state'].map(lambda s: df_to_list(renovation_states).index(s))
df_numeric['attributes/protected_area'] = df_numeric['attributes/protected_area'].map(lambda a: df_to_list(protected_area_options).index(a))
df_numeric['attributes/parking'] = df_numeric['attributes/protected_area'].map(lambda p: int(bool(p)))
df_numeric['financials/installmentPlan'] = df_numeric['financials/installmentPlan'].map(lambda ip: int(ip))

df_numeric

### Створення набору даних для семплів

In [ ]:
df_samples = df_numeric.copy()

df_samples = df_samples.drop(['price', 'pricePerUnit', 'attributes/num_apartments'], axis=1)

X = df_samples.values

X[0]

In [ ]:
X.shape

In [ ]:
df_samples

### Створення набору даних для міток

In [ ]:
y = df_numeric['price'].values

y[0]

In [ ]:
y.shape

### Нормалізація і стандартизація даних

In [ ]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X, y = shuffle(X, y)

In [ ]:
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_normalized = scaler_x.fit_transform(X)
y_normalized = scaler_y.fit_transform(y.reshape(-1, 1))

##### Збереження поточного scaler інстансу для подальшого використання

In [ ]:
# import joblib

# scaler_x_filename = "scaler_x.save" 
# scaler_y_filename = "scaler_y.save"

# joblib.dump(scaler_x, scaler_x_filename) 
# joblib.dump(scaler_y, scaler_y_filename) 

# Подальше завантаження
# scaler_x = joblib.load(scaler_x_filename) 
# scaler_y = joblib.load(scaler_y_filename) 

In [ ]:
X_normalized

In [ ]:
y_normalized

In [ ]:
y_normalized.shape

# Створення моделі

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mean_squared_error

In [ ]:
learning_rate = 0.0001

### Покращення швидкодії за допомогою GPU

In [ ]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print('Num GPUs available: ', len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
model = Sequential([
    Dense(units=16, input_shape=(14, ), activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=1, activation='linear'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

# Тренування моделі

Бібліотека Keras на цьому кроці також створює валідаційний набір даних за допомогою якого можна оцінити чи нейронна мережа не переходить в режим overfitting

In [ ]:
validation_split = 0.1
history = model.fit(x=X_normalized, y=y_normalized, validation_split=validation_split, batch_size=10, epochs=30, shuffle=True, verbose=2)

In [ ]:
plt.plot(history.history['loss'])
plt.title('Функція втрат від кількості епох')
plt.ylabel('Втрати')
plt.xlabel('Епохи')
plt.show()

# Передбачення моделі

### Вивести на екран таблицю значень для характеристики

In [ ]:
def display_characteristic(characteristic, column_name='Характеристика Новобудови'):
    return pd.DataFrame(data=df[characteristic].unique(), columns=[column_name])

In [ ]:
display_characteristic('attributes/class')

### Характеристики нерухомості

In [ ]:
residential_complex = 4 # residence
developer = 6 # developer
building_class = 2 # attributes/class
construction_technology = 0 # attributes/construction_technology
wall = 3 #attributes/walls
insulation = 0 # attributes/insulation
heating = 2 # attributes/heating
renovation_sate = 0 # attributes/state
protected_area = 2 # attributes/protected_area
parking = 0 # attributes/parking
installment_plan = 1 # financials/installmentPlan
installment_plan_term = 3 # financials/installmentPlanTerm
area = 45 # area
rooms = 1 # rooms

pred_inputs = np.array([[
    residential_complex,
    developer,
    building_class,
    construction_technology,
    wall,
    insulation,
    heating,
    renovation_sate,
    protected_area,
    parking,
    installment_plan,
    installment_plan_term,
    area,
    rooms
]])

X_to_predict_normalized = scaler_x.transform(pred_inputs)
Y_to_predict_normalized = scaler_y.transform(y.reshape(-1, 1))

X_to_predict_normalized

In [ ]:
y_hat = model.predict(X_to_predict_normalized.reshape(1, -1))
y_hat

In [ ]:
y_hat

In [ ]:
uah = scaler_y.inverse_transform(y_hat.reshape(1, -1))[0][-1]
print(f'Вартість нерухомості: {uah:,} грн')